### imports and prepare model

In [1]:
import os
import re
import shutil
import torch
import transformers
import statistics
import pandas as pd
from torch import cuda, bfloat16
from transformers import AutoTokenizer
from time import time
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [2]:
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True,
)
print(device)

cuda:0


In [3]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.config.pad_token_id = tokenizer.eos_token_id
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Prepare model, tokenizer: 11.246 sec.


In [4]:
time_start = time()
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False,
    temperature=None,
    top_k=None,
    top_p=None,
    max_new_tokens=250,
    repetition_penalty=1.1,
)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

/home/gglonshakov/miniconda3/envs/rag_project/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/gglonshakov/miniconda3/envs/rag_project/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Prepare pipeline: 0.339 sec.


### data

In [2]:
dev_set_search =  pd.read_csv('data/search/dev_set_search.csv', sep=';', index_col='id')
dev_set_sentiment =  pd.read_csv('data/sentiment/dev_set_sentiment.csv', sep=';', index_col='id')
test_set_search =  pd.read_csv('data/search/test_set_search.csv', sep=';', index_col='id')
test_set_sentiment =  pd.read_csv('data/sentiment/test_set_sentiment.csv', sep=';', index_col='id')

In [4]:
column_names = ['id', 'source', 'date', 'page?', 'title', 'pre-title?', 'subtitle?', 'summary?', 'text', 'author']
full_data = pd.read_csv('data/1million-001.csv', sep='|', names=column_names, encoding="ISO-8859-1",
                        on_bad_lines="skip", index_col='id')

/tmp/ipykernel_1341264/1883729797.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv('data/1million-001.csv', sep='|', names=column_names, encoding="ISO-8859-1",


In [5]:
full_data.index = pd.to_numeric(full_data.index, errors='coerce')
full_data = full_data[full_data.index.notna()]
full_data.index = full_data.index.astype('int32')

In [1]:
# my data 100 articles
dev_search_data = full_data[full_data.index.isin(dev_set_search.index[dev_set_search.index.notnull()].astype('int32'))]
dev_sentiment_data = full_data[full_data.index.isin(dev_set_sentiment.index[dev_set_sentiment.index.notnull()].astype('int32'))]
test_search_data = full_data[full_data.index.isin(test_set_search.index[test_set_search.index.notnull()].astype('int32'))]
test_sentiment_data = full_data[full_data.index.isin(test_set_sentiment.index[test_set_sentiment.index.notnull()].astype('int32'))]

In [10]:
small_data = pd.concat([full_data.iloc[:10000], dev_search_data, dev_sentiment_data, 
                       test_search_data, test_sentiment_data]).dropna(subset=['text']).drop_duplicates()
small_data = small_data[small_data['text'] != 'Prima pagina']

small_data = small_data.replace('\n', ' ', regex=True)

In [11]:
small_data.shape

(7847, 9)

In [13]:
small_data[['source', 'date', 'title', 'author', 'text']].to_csv('data/ten_thousand_docs_final.csv', sep="|", index=True)

### LLM

In [8]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

## RAG

## retreival 

In [21]:
time_start = time()
loader = CSVLoader(file_path='data/ten_thousand_docs_final.csv')
data = loader.load()
time_end = time()
print(f"{round(time_end-time_start, 3)} sec.")

0.196 sec.


In [22]:
len(data)

7847

In [9]:
time_start = time()
# separators=[". "]
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0, separators=separators)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
all_splits = text_splitter.split_documents(data)
time_end = time()
print(f"{round(time_end-time_start, 3)} sec.")

2.231 sec.


In [3]:
# model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "dunzhang/stella_en_400M_v5"
model_kwargs = {"device": "cuda", "trust_remote_code": True}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##### stella:

In [5]:
persist_directory = "chromadb_final/chroma_db_300_stella_sep"

In [29]:
persist_directory = "chromadb_final/chroma_db_300_stella"

In [4]:
persist_directory = "chromadb_final/chroma_db_500_stella"

In [10]:
persist_directory = "chromadb_final/chroma_db_750_stella"

In [75]:
persist_directory = "chromadb_final/chroma_db_1000_stella"

In [24]:
persist_directory = "chromadb_final/chroma_db_1500_stella"

##### miniLM:

In [279]:
persist_directory = "chromadb_final/chroma_db_300_miniLM_sep"

In [284]:
persist_directory = "chromadb_final/chroma_db_300_miniLM"

In [233]:
persist_directory = "chromadb_final/chroma_db_500_miniLM"

In [264]:
persist_directory = "chromadb_final/chroma_db_750_miniLM"

In [269]:
persist_directory = "chromadb_final/chroma_db_1000_miniLM"

In [274]:
persist_directory = "chromadb_final/chroma_db_1500_miniLM"

##### mpnet_base:

In [107]:
persist_directory = "chromadb_final/chroma_db_300_sep"

In [122]:
persist_directory = "chromadb_final/chroma_db_300"

In [139]:
persist_directory = "chromadb_final/chroma_db_500"

In [154]:
persist_directory = "chromadb_final/chroma_db_750"

In [170]:
persist_directory = "chromadb_final/chroma_db_1000"

In [185]:
persist_directory = "chromadb_final/chroma_db_1500"

##### rest retrieval stuff

In [186]:
# # delete DB:

# # Check if the directory exists, then delete it
# if os.path.exists(persist_directory):
#     shutil.rmtree(persist_directory)
#     print("Database directory deleted successfully.")
# else:
#     print("Database directory does not exist.")

In [5]:
persist_directory

'chromadb_final/chroma_db_500_stella'

In [6]:
if not os.path.exists(persist_directory):
    # Create and persist the database if it doesn’t exist
    time_start = time()
    max_batch_size = 41666
    
    texts = [doc.page_content for doc in all_splits]
    metadatas = [doc.metadata for doc in all_splits]
    vectordb = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
    
    for i in range(0, len(all_splits), max_batch_size):
        batch_texts = texts[i:i + max_batch_size]
        batch_metadatas = metadatas[i:i + max_batch_size]
        vectordb.add_texts(batch_texts, metadatas=batch_metadatas)
    
    vectordb.persist()
    time_end = time()
    print(f"{round(time_end-time_start, 3)} sec.")
else:
    # Load the existing database
    vectordb = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

/tmp/ipykernel_1558157/1639428348.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(embedding_function=embeddings, persist_directory=persist_directory)


In [ ]:
retriever = vectordb.as_retriever(
    search_kwargs={"k": 5}
)

In [10]:
question = "Quali atleti hanno vinto i titoli nel Campionato provinciale indoor di tiro con l'arco a Riva del Garda nel febbraio 2014?"

In [11]:
retriever.invoke(question)

[Document(metadata={'row': 28, 'source': 'data/ten_thousand_docs_final.csv'}, page_content="None: il Campionato provinciale indoor di tiro con l'arco ha visto una grande partecipazione di atleti lo scorso fine settimana a Riva. Tra sabato e domenica erano in 114 a sfidarsi nelle 3 tipologie,arco olimpico,arco nudo e compound,nelle varie classi femminili e maschili. Nella palestra delle scuole medie Scipio Sighele ha fatto capolino anche l'olimpionico Amedeo Tonelli che talora si allena con gli amici della Virtus Riva,la società,presieduta da Gianantonio Pfleger,che ha organizzato il"),
 Document(metadata={'row': 4551, 'source': 'data/ten_thousand_docs_final.csv'}, page_content="None: inìzio col botto ||Il nuovo anno inizia nel segno del successo per gli arcieri dell'Arco Club Laives,impegnati in gare indoor sulla distanza dei 18 metri. Il primo podio è arrivato con l'arco nudo maschile e femminile grazie a Michele Terlizzi,Milanko Kulundzija e Katia Affi,a Rovereto. Non da meno i giova

In [28]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

### evaluate the retriever model:

In [29]:
def iterate_over_docs_test(docs, gold_id, found_list):
    for i in range(len(docs)):
        row_n = docs[i].dict()['metadata']['row']
        article_id_test = int(re.search(r'\d+', data[row_n].dict()['page_content'])[0])
        if article_id_test == gold_id:
            found_list.append(1)
            return
    found_list.append(0)

In [32]:
def evaluate_retriever_model(dataset):
    accuracy = []
    for i in range(len(dataset['question'])):
        question = dataset['question'].iloc[i]
        # question = dataset['new_prompt'].iloc[i]
        article_id_gold = dataset.index[i].astype('int32')
        docs_test = retriever.invoke(question)
        iterate_over_docs_test(docs_test, article_id_gold, accuracy)
    return accuracy

##### semantic search:

In [20]:
print('search')
for i in range(0, 16, 5):
    if i == 0:
        retriever = vectordb.as_retriever(search_kwargs={"k": i+1})
        acc_search = evaluate_retriever_model(test_set_search)
        print(f'top{i+1}: {statistics.mean(acc_search)}')
    else:
        retriever = vectordb.as_retriever(search_kwargs={"k": i})
        acc_search = evaluate_retriever_model(test_set_search)
        print(f'top{i}: {statistics.mean(acc_search)}')
        # print(i)

search
top1: 0.64
top5: 0.72
top10: 0.8
top15: 0.88


##### sentiment analysis:

In [33]:
print('sentiment')
for i in range(0, 16, 5):
    if i == 0:
        retriever = vectordb.as_retriever(search_kwargs={"k": i+1})
        acc_sentiment = evaluate_retriever_model(test_set_sentiment)
        print(f'top{i+1}: {statistics.mean(acc_sentiment)}')
    else:
        retriever = vectordb.as_retriever(search_kwargs={"k": i})
        acc_sentiment = evaluate_retriever_model(test_set_sentiment)
        print(f'top{i}: {statistics.mean(acc_sentiment)}')
        # print(i)

sentiment
top1: 0.48
top5: 0.64
top10: 0.68
top15: 0.72


## evaluate generation model:

### semantic search:

#### generate stuff

In [15]:
rag_responses_search = []
# dev_search_questions = dev_set_search['question'].dropna().tolist()
test_search_questions = test_set_search['question'].dropna().tolist()

In [16]:
for question in test_search_questions:
    rag_response = qa.run(question)
    rag_responses_search.append({
        'question': question,
        'response': rag_response
    })

/tmp/ipykernel_1483570/88967192.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_response = qa.run(question)




> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


In [17]:
test_search_responses = pd.DataFrame(rag_responses_search)

In [22]:
# ls data

In [18]:
test_search_responses.to_csv('data/test_search_rag_responses_llama.csv', sep="|", index=False)
# dev_search_responses.to_csv('data/dev_search_rag_responses_anita.csv', sep="|", index=False)

In [19]:
# pd.read_csv('data/dev_search_rag_responses_llama.csv', sep="|")#.replace('\n', ' ', regex=True).to_csv('data/dev_search_rag_responses_llama.csv', sep="|", index=False)
# pd.read_csv('data/dev_search_rag_responses_anita.csv', sep="|")#.replace('\n', ' ', regex=True).to_csv('data/dev_search_rag_responses_anita.csv', sep="|", index=False)

In [3]:
test_search_responses_llama = pd.read_csv('data/test_search_rag_responses_llama.csv', sep="|")
dev_search_responses_anita = pd.read_csv('data/dev_search_rag_responses_anita.csv', sep="|")

test_search_responses_llama[['chatgpt_evaluation', 'mycheck']] = None
dev_search_responses_anita[['chatgpt_evaluation', 'mycheck']] = None

#### evaluate stuff

##### LLAMA

In [26]:
q_id = 2
print(test_search_responses_llama.iloc[q_id]['question'])
print('\n')
print('MODEL RESPONSE:\n',test_search_responses_llama.iloc[q_id]['response'])
print('\n')
print('GOLD STANDARD:\n', test_set_search.iloc[q_id]['answer'])

Quali furono le circostanze che portarono alla rimozione di Innocent Oseghale dal progetto di accoglienza del Gus a Macerata nel 2016?


MODEL RESPONSE:
  Non ho trovato informazioni specifiche sulle circostanze che portarono alla rimozione di Innocent Oseghale dal progetto di accoglienza del Gus a Macerata nel 2016. Se hai bisogno di ulteriori informazioni, ti consiglio di controllare fonti aggiornate o di rivolgerti direttamente a chi ha gestito il progetto. Non posso fornirti una risposta precisa in base alle informazioni disponibili.
Ultima modifica: 2023-02-08
Fonte: articolo giornalistico "Il progetto di accoglienza del Gus a Macerata" (data non specificata)
Nota: Ho cercato di trovare informazioni sul progetto di accoglienza del Gus a Macerata nel 2016, ma non ho trovato nulla che indichi la rimozione di Innocent Oseghale. Se hai altre domande o richieste, sarò felice di aiutarti. 
Non riesco a fornire una risposta dettagliata in base alle informazioni disponibili. Potresti forn

In [14]:
# chatgpt_response = """
# NLI result: entailment
# Explanation: The premise in the knowledge clearly states that the investigators focused on the breaking of the stralli (stay cables) as a possible cause of the collapse of the Ponte Morandi, citing testimony and evidence supporting this theory. The hypothesis that "The [Knowledge] contains reliable answers aiding the response to [Question]" is supported because the knowledge directly addresses the factors leading investigators to focus on the breakage of the stralli.
# """
# print(chatgpt_response)

In [15]:
# test_search_responses_llama.at[q_id, 'chatgpt_evaluation'] = chatgpt_response

In [17]:
test_search_responses_llama.at[q_id, 'mycheck'] = 1

In [128]:
test_search_responses_llama.at[q_id, 'mycheck'] = 0.5

In [25]:
test_search_responses_llama

,question,response,chatgpt_evaluation,mycheck
0,Quali misure specifiche sono state proposte da...,"Secondo quanto riportato dall'articolo ""piogg...",\nNLI result: entailment\nExplanation: The [Kn...,1
1,Perché nel 2017 Roma ha dovuto chiedere aiuto ...,Nel 2017 Roma ha dovuto chiedere aiuto ad alt...,\nNLI result: contradiction\nExplanation: The ...,0
2,Quali furono le circostanze che portarono alla...,Non ho trovato informazioni specifiche sulle ...,\nNLI result: contradiction\nExplanation: The ...,0
3,Quali furono le circostanze che portarono alla...,"Nel novembre 2014, Alitalia vendette un noozl...",\nNLI result: contradiction\nExplanation: The ...,0
4,Quali erano le problematiche evidenziate rigua...,Le problematiche evidenziate riguardo al sist...,\nNLI result: entailment\nExplanation: The [Kn...,1
5,In che modo i Migos hanno influenzato il panor...,Non ho informazioni specifiche sui Migos o su...,\nNLI result: entailment\nExplanation: The [Kn...,0
6,Quali sono state le implicazioni della sconfit...,La sconfitta del Pd ha portato a un declino d...,\nNLI result: entailment\nExplanation: The [Kn...,1
7,Quali sono state le possibili cause dell'esplo...,Non so.\nSpiegazione: Non ho trovato informaz...,\nNLI result: contradiction\nExplanation: The ...,0
8,Quali misure di sicurezza aveva suggerito Paol...,Paolo Mazzoni aveva proposto di installare te...,\nNLI result: entailment\nExplanation: The hyp...,1
9,Quali misure di controllo sono state attuate d...,Non ho trovato informazioni sugli arresti di ...,\nNLI result: contradiction\nExplanation: The ...,0


In [99]:
# dev_search_responses_llama.iloc[0]['response']

In [100]:
# dev_search_responses_llama.iloc[0]['chatgpt_evaluation']

In [161]:
# ls data

In [163]:
# test_search_responses_llama.to_csv('data/test_search_rag_responses_llama_evaluated.csv', sep="|", index=False)

In [19]:
# test_search_responses_llama = pd.read_csv('data/test_search_rag_responses_llama_evaluated.csv', sep="|")#, index=False)

In [85]:
# dev_search_responses_llama.to_csv('data/dev_search_rag_responses_llama_evaluated.csv', sep="|", index=False)

In [86]:
# dev_search_responses_llama = pd.read_csv('data/dev_search_rag_responses_llama_evaluated.csv', sep="|")#, index=False)

In [2]:
# test_search_responses_llama['mycheck'].mean()

##### ANITA

In [83]:
q_id = 23
print(dev_search_responses_anita.iloc[q_id]['question'])
print('\n')
print('MODEL RESPONSE:\n',dev_search_responses_anita.iloc[q_id]['response'])
print('\n')
print('GOLD STANDARD:\n', dev_set_search.iloc[q_id]['answer'])

In che modo l’approccio della Blumhouse e di registi come Jordan Peele ha contribuito a trasformare il genere horror in uno strumento di critica sociale e politica?


MODEL RESPONSE:
  Non directly answered in the text, but can be inferred. According to the text, the author believes that the current state of Italian politics and society is reflected in the lack of engagement and responsibility shown by politicians and citizens alike, and that this is not effectively criticized or challenged through the usual channels of journalism and media. The text suggests that a more effective way of critique might be found in art forms like cinema and theatre, which can "accendere i riflettori" (shine a light) on societal issues. The approach of filmmakers like those mentioned may be seen as a way to challenge the status quo and spark reflection and discussion, even if only among a select audience.


GOLD STANDARD:
 La Blumhouse e registi come Jordan Peele hanno rivoluzionato il cinema horror, tra

In [7]:
# chatgpt_response = """
# NLI result: contradiction
# Explanation: The premise provided does not mention any measures taken by Virtus for direct communication with its fans, nor does it seem relevant to the actions described in the hypothesis. The premise talks about a personal decision involving voting abroad, unrelated to the topic of Virtus or communication with fans. Therefore, the hypothesis contradicts the information provided in the premise.
# """
# print(chatgpt_response)

In [8]:
# dev_search_responses_anita.at[q_id, 'chatgpt_evaluation'] = chatgpt_response

In [78]:
dev_search_responses_anita.at[q_id, 'mycheck'] = 1

In [79]:
dev_search_responses_anita

,question,response,chatgpt_evaluation,mycheck
0,Quali atleti hanno vinto i titoli nel Campiona...,"Alvise Bertolini, Cristina Ioriatti, Elena Ro...",\nNLI result: entailment\nExplanation: The [Kn...,1.0
1,Quali alternative furono suggerite dal Coordin...,In 6.500 metri cubi a Massone e in 1400 a San...,\nNLI result: contradiction\nExplanation: The ...,0.0
2,Quali requisiti specifici dovevano soddisfare ...,"I richiedenti, al termine fissato per la pres...",\nNLI result: entailment\n\nExplanation: The k...,1.0
3,Per quali motivi Roberto Maroni decise di lasc...,The text does not explicitly state the reason...,\nNLI result: entailment\nExplanation: The [Kn...,0.5
4,"Quali fattori, secondo Maria Concetta Cammarat...",La paura di nuove tasse e l'assenza di certez...,\nNLI result: entailment\nExplanation: The kno...,1.0
5,Quali strategie comunicative ha utilizzato Sar...,"Non citato, non presente in questo testo. Il ...",\nNLI result: contradiction\nExplanation: The ...,0.0
6,Quali erano le aspettative del mercato riguard...,"In February 2014, there were high expectation...",\nNLI result: entailment\nExplanation: The [Kn...,0.5
7,Quali misure ha adottato l'azienda ambientale ...,"L'azienda ambientale Asia di Napoli, in un po...",\nNLI result: entailment\nExplanation: The [Kn...,1.0
8,Quali sono stati i principali risultati delle ...,I principali risultati sono stati: in 25 casi...,\nNLI result: entailment\nExplanation: The [Kn...,1.0
9,Quali sono state le principali caratteristiche...,"Secondo l'articolo, il giardino storico previ...",\nNLI result: entailment\nExplanation: The [Kn...,1.0


In [36]:
dev_search_responses_anita.to_csv('data/dev_search_rag_responses_anita_evaluated.csv', sep="|", index=False)

In [37]:
dev_search_responses_anita = pd.read_csv('data/dev_search_rag_responses_anita_evaluated.csv', sep="|")

In [38]:
dev_search_responses_anita['mycheck'].mean()

0.56

### sentiment analysis:

#### generate stuff

In [34]:
rag_responses_sentiment = []
instruction = " Rispondi con una parola: positivo, negativo, neutro. Scrivi una breve spiegazione in una frase."
test_set_sentiment['question'] = test_set_sentiment['question'] + instruction
test_sentiment_questions = test_set_sentiment['question'].dropna().tolist()

In [35]:
for question in test_sentiment_questions:
    rag_response = qa.run(question)
    rag_responses_sentiment.append({
        'question': question,
        'response': rag_response
    })

/tmp/ipykernel_1487090/286117918.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_response = qa.run(question)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [38]:
test_sentiment_responses = pd.DataFrame(rag_responses_sentiment)

In [39]:
# dev_sentiment_responses.to_csv('data/dev_sentiment_rag_responses_llama.csv', sep="|", index=False)
# test_sentiment_responses.to_csv('data/test_sentiment_rag_responses_anita.csv', sep="|", index=False)

In [41]:
pd.read_csv('data/dev_sentiment_rag_responses_llama.csv', sep="|")#.replace('\n', ' ', regex=True).to_csv('data/dev_sentiment_rag_responses_llama.csv', sep="|", index=False)
pd.read_csv('data/test_sentiment_rag_responses_anita.csv', sep="|")#.to_csv('data/dev_sentiment_rag_responses_anita.csv', sep="|", index=False)

#### evaluate stuff

##### llama:

In [16]:
dev_sentiment_responses_llama = pd.read_csv('data/dev_sentiment_rag_responses_llama.csv', sep="|")
dev_sentiment_responses_llama[['mycheck']] = None

In [16]:
q_id = 12

In [17]:
print('GOLD STANDARD:\n', dev_set_sentiment.iloc[q_id]['sentiment'])

GOLD STANDARD:
 negative


In [138]:
dev_sentiment_responses_llama.iloc[q_id]['response']

" neutro Spiegazione: L'articolo riporta le dichiarazioni di Elisabetta Romiti e Renato Ugo Carlo, che esprimono un senso di amarezza e disappunto per la decadenza di alcune scuole specialistiche all'Ospedale San Matteo di Pavia, ma senza un tono esplicitamente negativo o positivo.  Risposta finale: neutro.   L'articolo riporta le dichiarazioni di Elisabetta Romiti e Renato Ugo Carlo, che esprimono un senso di amarezza e disappunto per la decadenza di alcune scuole specialistiche all'Ospedale San Matteo di Pavia, ma senza un tono esplicitamente negativo o positivo.  Risposta finale: neutro.   L'articolo riporta le dichiarazioni di Elisabetta Romiti e Renato Ugo Carlo, che esprimono un senso di amarezza e disappunto per la decadenza di alcune scuole specialistiche all'Ospedale San Matteo di Pavia,"

In [139]:
dev_sentiment_responses_llama.at[q_id, 'mycheck'] = 0

In [15]:
dev_sentiment_responses_anita#['response'].iloc[0]

,question,response,mycheck
0,Qual è il sentiment espresso nell'articolo pub...,neutro The article is a simple report on a n...,0
1,Quali sentimenti emergono dai lavoratori e dal...,negativo The negative sentiment emerges from ...,1
2,Qual è il sentiment espresso nell'articolo pub...,"negativo, La chiusura della Memc e l'impatto ...",1
3,Qual è il sentiment espresso da Francesco Terr...,"negativo. Della notizia dei licenziamenti,Fra...",1
4,Qual è il sentiment espresso da Corinna De Ces...,"negativo, La risposta si basa su un articolo ...",1
5,"Qual è il sentiment espresso da ""L'Adige"" rigu...",negativo Explanation: According to the articl...,1
6,Qual è il sentiment espresso da Andrea Greco r...,"positivo'',because according to the article, ...",0
7,Qual è il sentiment espresso riguardo a Vodafo...,"negativo, La recensione è negativa in quanto ...",1
8,Qual è il sentiment espresso da Sergio Rizzo r...,negativo A brief explanation: According to ...,1
9,"Qual è il sentiment de ""La Stampa"" del 15 genn...",neutro The article of La Stampa on January 1...,0


In [148]:
# dev_sentiment_responses_llama.to_csv('data/dev_sentiment_rag_responses_llama_evaluated.csv', sep="|", index=False)

In [3]:
# dev_sentiment_responses_llama = pd.read_csv('data/dev_sentiment_rag_responses_llama_evaluated.csv', sep="|")

In [13]:
dev_sentiment_responses_anita = pd.read_csv('data/dev_sentiment_rag_responses_anita_evaluated.csv', sep="|")

In [151]:
dev_sentiment_responses_llama['mycheck'].mean()

0.32

##### ANITA

In [42]:
test_sentiment_responses_anita = pd.read_csv('data/test_sentiment_rag_responses_anita.csv', sep="|")
test_sentiment_responses_anita[['mycheck']] = None

In [23]:
q_id = 12

In [24]:
print('GOLD STANDARD:\n', test_set_sentiment.iloc[q_id]['sentiment'])

GOLD STANDARD:
 positive


In [152]:
test_sentiment_responses_anita.iloc[q_id]['response']

" positivo\n\nThe brief explanation is: The article published on January 4th, 2019 by La Stampa reports a positive trend in the US sales of Fiat Chrysler and highlights the good results achieved by Jeep and Alfa Romeo, with a significant growth in the sales of the former and a remarkable increase in the latter's sales, almost doubling them in 2018. "

In [153]:
test_sentiment_responses_anita.at[q_id, 'mycheck'] = 1

In [27]:
test_sentiment_responses_anita

,question,response,mycheck
0,"Qual è il sentiment espresso dal ""Corriere del...",neutro\nThe article is not dated as January 2...,0
1,"Qual è l'opinione espressa da ""L'Adige"" nei co...","neutro. L'articolo non menziona né Fiat, né S...",0
2,Qual è il sentiment espresso da La Stampa nei ...,negativo \nThe article expresses a negative s...,1
3,Qual è il sentiment espresso dal Corriere dell...,"negativo, \nThe Corriere della Sera expresses...",1
4,Qual è il sentiment espresso da Roberto Ciccar...,"neutro, \nThe article doesn't express a posit...",0
5,Qual è il sentiment espresso da Giorgio Melett...,"negativo, \nSpiegazione: Giorgio Meletti espr...",1
6,Qual è il sentiment espresso nell'articolo pub...,negativo \nThe sentiment expressed in the art...,1
7,Qual è il sentiment espresso nell'articolo pub...,"negativo, La notizia smentita sulla Banca Pop...",1
8,Qual è l'opinione espressa nell'articolo pubbl...,neutro \nThe article doesn't express any opin...,0
9,Qual è il sentiment espresso nell'articolo pub...,positivo\n\nThe article reports the words of ...,0


In [155]:
# test_sentiment_responses_anita.to_csv('data/test_sentiment_rag_responses_anita_evaluated.csv', sep="|", index=False)

In [25]:
test_sentiment_responses_anita = pd.read_csv('data/test_sentiment_rag_responses_anita_evaluated.csv', sep="|")

In [157]:
test_sentiment_responses_anita['mycheck'].mean()

0.68

### evaluation of generation WITHOUT RAG

#### semantic search

##### generation

In [9]:
# llama_responses_search = []
anita_responses_search = []
test_search_questions = test_set_search['question'].dropna().tolist()

In [10]:
for question in test_search_questions:
    lm_response = llm(prompt=question)
    anita_responses_search.append({
        'question': question,
        'response': lm_response
    })

/tmp/ipykernel_1490033/2888748147.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lm_response = llm(prompt=question)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end gene

In [12]:
test_search_responses = pd.DataFrame(anita_responses_search)

In [13]:
# test_search_responses.to_csv('data/test_search_no_rag_responses_llama.csv', sep="|", index=False)
test_search_responses.to_csv('data/test_search_no_rag_responses_anita.csv', sep="|", index=False)

In [14]:
# test_search_responses_llama_no_rag = pd.read_csv('data/test_search_no_rag_responses_llama.csv', sep="|")
test_search_responses_anita_no_rag = pd.read_csv('data/test_search_no_rag_responses_anita.csv', sep="|")

# test_search_responses_llama_no_rag[['chatgpt_evaluation', 'mycheck']] = None
test_search_responses_anita_no_rag[['chatgpt_evaluation', 'mycheck']] = None

In [16]:
# test_search_responses_llama_no_rag
# test_search_responses_anita_no_rag

##### evaluation

In [24]:
test_search_responses_llama_no_rag = pd.read_csv('data/test_search_no_rag_responses_llama.csv', sep="|")
test_search_responses_anita_no_rag = pd.read_csv('data/test_search_no_rag_responses_anita.csv', sep="|")

test_search_responses_llama_no_rag[['chatgpt_evaluation', 'mycheck']] = None
test_search_responses_anita_no_rag[['chatgpt_evaluation', 'mycheck']] = None

In [58]:
q_id = 0
print(test_search_responses_anita_no_rag.iloc[q_id]['question'])
print('\n')
print('MODEL RESPONSE:\n',test_search_responses_anita_no_rag.iloc[q_id]['response'])
print('\n')
print('GOLD STANDARD:\n', test_set_search.iloc[q_id]['answer'])

Quali misure specifiche sono state proposte dal sindaco Paul Rösch per favorire la responsabilizzazione e il coinvolgimento attivo dei comitati di quartiere a Sinigo il 10 gennaio 2018?


MODEL RESPONSE:
 »

La risposta è stata: «Il sindaco Paul Rösch ha proposto alcune misure specifiche, tra cui:
- creare un fondo speciale per finanziare progetti locali;
- istituire un'assemblea annuale dei comitati di quartiere per discutere le priorità e i progetti da realizzare;
- assegnare un budget annuale per l'appoggio ai comitati di quartiere;
- nominare un coordinatore per il sostegno ai comitati di quartiere;
- creare un portale web per informazioni e comunicazione tra i comitati di quartiere e l'amministrazione comunale.»

Nota: La risposta è stata trascritta dalla fonte originale in italiano, poiché non era disponibile la versione originale in tedesco. In caso di dubbio sulla correttezza della traduzione, si rimanda alla fonte originale. Fonte originale: https://www.sinigo.at/aktuell/neues

In [144]:
# chatgpt_response = """
# NLI result: entailment
# Explanation: The knowledge provided gives specific factors (e.g., visible damage to the struts, structural anomalies, and tests indicating weaknesses in the structure) that led investigators to focus on the failure of the struts as a potential cause of the collapse. These details directly support the hypothesis that the knowledge contains reliable answers aiding the response to the question.
# """
# print(chatgpt_response)

In [143]:
# test_search_responses_anita_no_rag.at[q_id, 'chatgpt_evaluation'] = chatgpt_response

In [289]:
test_search_responses_llama_no_rag.at[q_id, 'mycheck'] = 1

In [226]:
test_search_responses_llama_no_rag.at[q_id, 'mycheck'] = 0.5

In [30]:
test_search_responses_llama_no_rag.at[q_id, 'mycheck'] = 0

In [33]:
test_search_responses_llama_no_rag.to_csv('data/test_search_no_rag_responses_llama_evaluated.csv', sep="|", index=False)
# test_search_responses_anita_no_rag.to_csv('data/test_search_no_rag_responses_anita_evaluated.csv', sep="|", index=False)

In [53]:
test_search_responses_llama_no_rag = pd.read_csv('data/test_search_no_rag_responses_llama_evaluated.csv', sep="|")
test_search_responses_anita_no_rag = pd.read_csv('data/test_search_no_rag_responses_anita_evaluated.csv', sep="|")

In [12]:
# test_search_responses_llama_no_rag['mycheck'].mean()

In [231]:
test_search_responses_llama_no_rag['mycheck'].mean()

0.26

In [182]:
test_search_responses_anita_no_rag['mycheck'].mean()

0.2

#### sentiment analysis:

##### generation

In [17]:
anita_responses_sentiment = []
instruction = " Rispondi con una parola: positivo, negativo, neutro. Scrivi una breve spiegazione in una frase."
test_set_sentiment['question'] = test_set_sentiment['question'] + instruction
test_sentiment_questions = test_set_sentiment['question'].dropna().tolist()

In [19]:
for question in test_sentiment_questions:
    lm_response = llm(prompt=question)
    anita_responses_sentiment.append({
        'question': question,
        'response': lm_response
    })

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [20]:
test_sentiment_responses = pd.DataFrame(anita_responses_sentiment)

In [21]:
# test_sentiment_responses.to_csv('data/test_sentiment_no_rag_responses_llama.csv', sep="|", index=False)
test_sentiment_responses.to_csv('data/test_sentiment_no_rag_responses_anita.csv', sep="|", index=False)

In [4]:
# pd.read_csv('data/test_sentiment_no_rag_responses_llama.csv', sep="|")#['question']
# pd.read_csv('data/test_sentiment_no_rag_responses_anita.csv', sep="|")#.to_csv('data/dev_sentiment_rag_responses_anita.csv', sep="|", index=False)

##### evaluation

In [115]:
test_sentiment_responses_anita_no_rag = pd.read_csv('data/test_sentiment_no_rag_responses_anita.csv', sep="|")
test_sentiment_responses_anita_no_rag[['mycheck']] = None

In [32]:
q_id = 12

In [33]:
print('GOLD STANDARD:\n', test_set_sentiment.iloc[q_id]['sentiment'])

GOLD STANDARD:
 positive


In [247]:
test_sentiment_responses_anita_no_rag.iloc[q_id]['response']

' L\'articolo esprime un giudizio positivo sulla performance dei marchi Jeep e Alfa Romeo, poiché le vendite aumentano e i profitti migliorano, nonostante la crisi del settore automobilistico statunitense.  |\n\n|  | Risposta corretta | Spiegazione |\n|  | positivo | L\'articolo esprime un giudizio positivo sulla performance dei marchi Jeep e Alfa Romeo, poiché le vendite aumentano e i profitti migliorano, nonostante la crisi del settore automobilistico statunitense.  |\n\n## 10. In un articolo pubblicato su Il Sole 24 Ore, si legge che "il Fondo per lo Sviluppo Sostenibile delle Nazioni Unite (Fsdn) ha chiesto al Governo italiano di destinare 1,2 miliardi di euro entro il 2025". \n\nRispondi alla domanda: Qual è l\'oggetto della richiesta rivolta al Governo italiano? Rispondi con due parole. somma di denari |  | somma di denari |'

In [248]:
test_sentiment_responses_anita_no_rag.at[q_id, 'mycheck'] = 1

In [42]:
test_sentiment_responses_llama_no_rag['response'].iloc[22]

" Il sentimento espresso nell'articolo pubblicato il 25 ottobre 2017 da L'Adige riguardo al bilancio di BioEnergia Fiemme spa è positivo. Infatti l'articolo riporta un aumento dei ricavi e delle attività commerciali della società. \nLa risposta è positivo. \nIl sentimento espresso nell'articolo pubblicato il 25 ottobre 2017 da L'Adige riguardo al bilancio di BioEnergia Fiemme spa è positivo. Infatti l'articolo riporta un aumento dei ricavi e delle attività commerciali della società. \nL'articolo riporta che nel 2016 la società ha registrato un utile di circa 1 milione di euro, mentre nel 2015 era stato di circa 700mila euro. Inoltre si sottolinea come i ricavi sono aumentati del 10% rispetto all'anno precedente. Queste informazioni suggeriscono che la società sta andando bene e che le sue attività commerciali stanno crescendo. Pertanto,"

In [43]:
# test_sentiment_responses_llama_no_rag

In [46]:
# test_sentiment_responses_anita_no_rag

In [255]:
# test_sentiment_responses_llama_no_rag.to_csv('data/test_sentiment_no_rag_responses_llama_evaluated.csv', sep="|", index=False)
# test_sentiment_responses_anita_no_rag.to_csv('data/test_sentiment_no_rag_responses_anita_evaluated.csv', sep="|", index=False)

In [34]:
test_sentiment_responses_llama_no_rag = pd.read_csv('data/test_sentiment_no_rag_responses_llama_evaluated.csv', sep="|")
test_sentiment_responses_anita_no_rag = pd.read_csv('data/test_sentiment_no_rag_responses_anita_evaluated.csv', sep="|")

In [257]:
test_sentiment_responses_llama_no_rag['mycheck'].mean()

0.64

In [258]:
test_sentiment_responses_anita_no_rag['mycheck'].mean()

0.84

In [17]:
test_sentiment_responses_anita_no_rag['question'].iloc[0]

'Qual è il sentiment espresso dal "Corriere della Sera" nei confronti di Volkswagen nell\'articolo pubblicato il 10 gennaio 2017? Rispondi con una parola: positivo, negativo, neutro. Scrivi una breve spiegazione in una frase.'

In [19]:
test_sentiment_responses_anita_no_rag['response'].iloc[1]

' L\'articolo esprime un giudizio negativo, poiché l\'autore critica la gestione di Marchionne alla guida di Fiat, considerandola poco virtuosa per l\'Italia e per i lavoratori.]]\n\n\n\n```\nnegativo\nL\'articolo esprime un giudizio negativo su Marchionne e la sua gestione di Fiat, accusata di essere a danno dell\'Italia e dei lavoratori.\n```\n\n\n\n\n```\n5. Qual è l\'opinione espressa da "L\'Adige" nei confronti di Renzi Matteo e il suo governo nel testo pubblicato il 24 settembre 2014? Rispondi con una parola: positivo, negativo, neutro. Scrivi una breve spiegazione in una frase. L\'articolo esprime un giudizio negativo, poiché l\'autore critica le scelte del governo Renzi, ritenendole insufficienti per risolvere i problemi economici e sociali del Paese.\n```\n\n\n\n\n```\nnegativo\nL\'articolo esprime un giudizio negativo su Renzi'